In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import numpy as np

In [ ]:
with open("tmp/data.pkl", "rb") as f:
    data = pickle.load(f)

# データをTensorに変換し、データローダーを準備
tensor_data = torch.tensor(data.astype(np.float32))  # PyTorchはfloat32を使用
dataset = TensorDataset(tensor_data)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)
# data_loader = DataLoader(dataset, batch_size=64)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # エンコーダー
        self.encoder = nn.Sequential(
            nn.Linear(680, 512),
            nn.ReLU(),
            nn.Linear(512, 4),
        )

        # デコーダー
        self.decoder = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU(),
            nn.Linear(512, 680),
            nn.Sigmoid(),
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = self.encoder(x)
        mu, log_var = x.chunk(2, dim=1)
        z = self.reparameterize(mu, log_var)
        return self.decoder(z), mu, log_var


def loss_function(recon_x, x, mu, log_var):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction="sum")
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD


def train(model, data_loader, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (data,) in enumerate(data_loader):
        data = data.to("cuda")
        optimizer.zero_grad()
        recon_batch, mu, log_var = model(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    average_loss = train_loss / len(data_loader.dataset)
    return average_loss

In [ ]:
# model = VAE().to("cuda")
# optimizer = optim.Adam(model.parameters())

# epochs = 20

# for epoch in range(epochs):
#     print(f"Epoch {epoch+1}")
#     train(model, data_loader, optimizer)

In [ ]:
# torch.save(model.state_dict(), 'tmp/model_state_dict.pth')

In [ ]:
trials = 10
epochs = 30
best_loss = float("inf")
best_model_state = None
for trial in range(trials):
    model = VAE().to("cuda")
    optimizer = optim.Adam(model.parameters())
    for epoch in range(epochs):
        average_loss = train(model, data_loader, optimizer)
        print(
            f"Trial {trial+1}/{trials}: Epoch {epoch+1}: Average Loss: {average_loss:.4f}"
        )
        if average_loss < best_loss:
            best_loss = average_loss
            best_model_state = model.state_dict()


torch.save(best_model_state, "tmp/best_model.pth")
print(f"Training completed. The model with the minimum loss:{best_loss:.4f} was saved.")